In [1]:
import os
from databricks.connect import DatabricksSession

# if running on a local IDE
spark = DatabricksSession.builder.remote(serverless=True).getOrCreate()

In [2]:
import os
import mlflow

# TODO make sure you update the config file before this

configs = mlflow.models.ModelConfig(development_config="./config.yaml")
databricks_configs = configs.get("databricks")
agent_configs = configs.get("agent")

CATALOG = databricks_configs["catalog"]
SCHEMA = databricks_configs["schema"]
UC_MODEL = databricks_configs["model"]
WORKSPACE_URL = databricks_configs["workspace_url"]
SQL_WAREHOUSE_ID = databricks_configs["sql_warehouse_id"]
MLFLOW_EXPERIMENT_NAME = databricks_configs["mlflow_experiment_name"]
AGENT_NAME = agent_configs["name"]

In [3]:
import mlflow

mlflow.set_registry_uri("databricks-uc")
mlflow.set_tracking_uri("databricks")

experiment_path = f"/Workspace/Users/david.huang@databricks.com/client-demos/agent/oai-response-agent/{MLFLOW_EXPERIMENT_NAME}"

try:
    experiment_id = mlflow.create_experiment(name=experiment_path)
    mlflow.set_experiment(experiment_path)
    print(f"Created and set new experiment: {experiment_path} (ID: {experiment_id})")
except mlflow.exceptions.RestException as e:
    if "already exists" in str(e):
        print(f"Set to existing MLflow experiment: {experiment_path}")
        mlflow.set_experiment(experiment_path)
    else:
        raise e

Set to existing MLflow experiment: /Workspace/Users/david.huang@databricks.com/client-demos/agent/oai-response-agent/oai_gdr_agent_exp


## Load & test agent

Make sure you go to the MLflow experiment to look at trace data as you develop & debug the agent.

In [7]:
%run ./agent.py

2025/09/09 12:44:57 WARNING mlflow.pyfunc: You have manually traced predict with @mlflow.trace, but this is unnecessary with ResponsesAgent subclasses. You can remove the @mlflow.trace decorator and it will be automatically traced.
2025/09/09 12:44:57 WARNING mlflow.pyfunc: You have manually traced predict_stream with @mlflow.trace, but this is unnecessary with ResponsesAgent subclasses. You can remove the @mlflow.trace decorator and it will be automatically traced.


In [9]:
result = AGENT.predict(
    {
        "input": [
            {
                "role": "user",
                "content": "Compare and contrast the annual net income growth in the past 10 years between BAC and AXP",
            }
        ]
    }
)

print(result.model_dump()["output"][-1]["content"][0]["text"])

# Comparison of Annual Net Income Growth: BAC vs AXP (2013-2022)

Based on the analysis of the past 10 years of net income data for Bank of America (BAC) and American Express (AXP), here's a comprehensive comparison:

## Overall Performance

1. **Scale Difference**: BAC operates at a significantly larger scale than AXP, with an average annual net income of $20.13 billion compared to AXP's $5.69 billion.

2. **Total Growth**: From 2013 to 2022, BAC experienced much stronger total growth at 140.82%, while AXP grew by 40.21%.

3. **Compound Annual Growth Rate (CAGR)**: BAC achieved a CAGR of 10.26%, substantially outperforming AXP's 3.83%.

## Growth Patterns

1. **Volatility**: Both companies showed high volatility in their growth rates, but BAC experienced more extreme fluctuations (standard deviation of 92.44% vs. 73.99% for AXP).

2. **Positive Growth Years**:
   - BAC had 6 years of positive growth (2013, 2015, 2016, 2017, 2018, 2021)
   - AXP had 5 years of positive growth (2013, 20